In [1]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import shutil
import cv2
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"

In [3]:
model = hub.load(SAVED_MODEL_PATH)

Downloaded https://tfhub.dev/captain-pool/esrgan-tf2/1, Total size: 20.60MB



In [4]:
def preprocess_image(image_path):
  """ Loads image from path and preprocesses to make it model ready
      Args:
        image_path: Path to the image file
  """
  hr_image = tf.image.decode_image(tf.io.read_file(image_path))
  if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
  hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
  hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
  hr_image = tf.cast(hr_image, tf.float32)
  return tf.expand_dims(hr_image, 0)

def save_image(image, filename):
  """
    Saves unscaled Tensor Images.
    Args:
      image: 3D image tensor. [height, width, channels]
      filename: Name of the file to save.
  """
  if not isinstance(image, Image.Image):
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  image.save("%s.png" % filename)
  print("Saved as %s.png" % filename)

In [5]:
%matplotlib inline
def plot_image(image, title=""):
  """
    Plots images from image tensors.
    Args:
      image: 3D image tensor. [height, width, channels].
      title: Title to display in the plot.
  """
  image = np.asarray(image)
  image = tf.clip_by_value(image, 0, 255)
  image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  plt.imshow(image)
  plt.axis("off")

In [6]:
# Defining helper functions
def downscale_image(image):
  """
      Scales down images using bicubic downsampling.
      Args:
          image: 3D or 4D tensor of preprocessed image
  """
  image_size = []
  if len(image.shape) == 3:
    image_size = [image.shape[1], image.shape[0]]
  else:
    raise ValueError("Dimension mismatch. Can work only on single image.")

  image = tf.squeeze(
      tf.cast(
          tf.clip_by_value(image, 0, 255), tf.uint8))

  lr_image = np.asarray(
    Image.fromarray(image.numpy())
    .resize([image_size[0] // 4, image_size[1] // 4],
              Image.BICUBIC))

  lr_image = tf.expand_dims(lr_image, 0)
  lr_image = tf.cast(lr_image, tf.float32)
  return lr_image

In [7]:
source_folder = '/kaggle/input/pad-uefs-20/PAD-UFES-20_images_part_2'
destination_folder = '/kaggle/working/PAD-UFES-20_images_part_2'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)


In [8]:
import os
import cv2
import numpy as np
import tensorflow as tf
import time

# Assuming you have defined your preprocess_image, downscale_image, and model functions
# ...

i=0

for filename in os.listdir(source_folder):
    
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        
        source_image_path = os.path.join(source_folder, filename)
        destination_image_path = os.path.join(destination_folder, os.path.splitext(filename)[0])

        hr_image = preprocess_image(source_image_path)
        lr_image = downscale_image(tf.squeeze(hr_image))
        start = time.time()
        fake_image = model(lr_image)
        fake_image = tf.squeeze(fake_image)
        print("Time Taken: %f" % (time.time() - start))
        print("Image Number: %f" % (i+1))
        i = i+1

        save_image(fake_image,destination_image_path)


Time Taken: 7.652348
Image Number: 1.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_2/PAT_650_3858_329.png
Time Taken: 1.829439
Image Number: 2.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_2/PAT_766_1444_52.png
Time Taken: 1.776640
Image Number: 3.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_2/PAT_806_1520_60.png
Time Taken: 2.355808
Image Number: 4.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_2/PAT_677_1350_358.png
Time Taken: 3.276668
Image Number: 5.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_2/PAT_856_1626_630.png
Time Taken: 1.530787
Image Number: 6.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_2/PAT_741_1400_780.png
Time Taken: 3.344665
Image Number: 7.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_2/PAT_754_1429_380.png
Time Taken: 1.342194
Image Number: 8.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_2/PAT_778_1471_835.png
Time Taken: 1.792674
Image Number: 9.000000
Saved as /kaggle/working/PAD-U

In [15]:

# import shutil
# import os

 
# # removing directory
# shutil.rmtree("/kaggle/working/PAD-UFES-20_images_part_22forzip")

In [16]:
source_folder = '/kaggle/working/PAD-UFES-20_images_part_2'
destination_folder = '/kaggle/working/PAD-UFES-20_images_part_22forzip'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

In [17]:
import os
import cv2
import numpy as np
import tensorflow as tf
import time

# Assuming you have defined your preprocess_image, downscale_image, and model functions
# ...

i=0

for filename in os.listdir(source_folder):
    
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        
        source_image_path = os.path.join(source_folder, filename)
        destination_image_path = os.path.join(destination_folder, os.path.splitext(filename)[0])

        # Load the image
        image = cv2.imread(source_image_path)

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Apply Canny edge detection
        edges = cv2.Canny(blurred, 30, 150)

        # Apply morphological operations (dilation and erosion)
        kernel = np.ones((5, 5), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=1)
        edges = cv2.erode(edges, kernel, iterations=1)

        # Create a hair mask by thresholding
        _, hair_mask = cv2.threshold(edges, 30, 255, cv2.THRESH_BINARY)

        # Inpainting to remove hair artifacts
        inpainting_result = cv2.inpaint(image, hair_mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

        # Display the inpainting result
        # cv2.imwrite("/kaggle/working/inpainted_image.jpg", inpainting_result)
        image_rgb = cv2.cvtColor(inpainting_result, cv2.COLOR_BGR2RGB)

        print("Image Number: %f" % (i+1))
        i = i+1

        save_image(image_rgb,destination_image_path)

Image Number: 1.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_683_1297_195.png
Image Number: 2.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_691_1311_890.png
Image Number: 3.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_844_1605_308.png
Image Number: 4.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_851_4519_488.png
Image Number: 5.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_658_1266_149.png
Image Number: 6.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_903_1720_647.png
Image Number: 7.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_598_1137_888.png
Image Number: 8.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_756_1496_480.png
Image Number: 9.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_868_1657_698.png
Image Number: 10.000000
Saved as /kaggle/working/PAD-UFES-20_images_part_22forzip/

In [18]:
!zip -r filePart22.zip /kaggle/working/PAD-UFES-20_images_part_22forzip

  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/ (stored 0%)
  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_683_1297_195.png (deflated 0%)
  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_691_1311_890.png (deflated 0%)
  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_844_1605_308.png (deflated 0%)
  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_851_4519_488.png (deflated 0%)
  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_658_1266_149.png (deflated 0%)
  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_903_1720_647.png (deflated 0%)
  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_598_1137_888.png (deflated 0%)
  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_756_1496_480.png (deflated 0%)
  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_868_1657_698.png (deflated 0%)
  adding: kaggle/working/PAD-UFES-20_images_part_22forzip/PAT_963_1819_298.png (deflated 0%)

<a href="filePart22.zip"> Download File </a>

<a href="filePart1.zip"> Download File </a>